In [7]:
### 测试stationary函数
import os
from util import stationary
import numpy as np
# 创建一个简单的过渡矩阵
A = np.array([[0.4, 0.6],
              [0.2, 0.8]])

# 调用函数
result = stationary(A)
result

/root/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1272: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


array([0.5, 1.5])

In [6]:
!pip install torch_geometric>=2.2.0

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
